In [13]:
import string
import sys
import itertools
import random
from helpers.players import Player
from functools import reduce
from operator import mul

## Setup

If most people have 2-3 roles available, the number of available combinations can grow really high. For example, if we have 24 players and each can have all three roles, the number is `3 ** 24` which equals `282429536481`.

In this case, since everyone can do anything, we could simply just shuffle the `players` list and pick first `n` as tanks, following `n` as healers and rest as DDs.

Thus, we can introduce an early stopping condition.

## Pick 20 players

Let's start by generating 20 players who **all** can tank and DD, but only about half can heal.

In [113]:
# How many players to create
n = 4 * 5

# ASCII Letters
a_z = string.ascii_letters

# Container
players = []

# Add the first players as can-do-all
players.append(Player(name="primus", can_tank=True, can_dd=True, can_heal=True))

for i in range(n - 1):
    p = Player(
        name=a_z[i], 
        can_tank=True, 
        can_dd=True, 
        can_heal=bool(random.getrandbits(1)))
    players.append(p)

## Find out n of combinations

In [114]:
[len(p.roles) or 1 for p in players]

[3, 2, 3, 3, 3, 2, 2, 3, 2, 3, 2, 2, 3, 2, 2, 3, 2, 3, 3, 2]

In [115]:
n_combinations = reduce(mul, [len(p.roles) or 1 for p in players])

print(f"[INFO] The players can have {n_combinations:,} different role combinations")

[INFO] The players can have 60,466,176 different role combinations


## Find group size

Depending on how many players we have and what roles they have available, we can create `n_groups` number of groups. For example, if we had 100 players but only two could tank, the `n_groups` would be `min(n_tanks)`.

In [116]:
# List of lists. Each list element contains one player's roles as strings.
player_roles = [p.roles for p in players]

player_roles

[['tank', 'heal', 'dd'],
 ['tank', 'dd'],
 ['tank', 'heal', 'dd'],
 ['tank', 'heal', 'dd'],
 ['tank', 'heal', 'dd'],
 ['tank', 'dd'],
 ['tank', 'dd'],
 ['tank', 'heal', 'dd'],
 ['tank', 'dd'],
 ['tank', 'heal', 'dd'],
 ['tank', 'dd'],
 ['tank', 'dd'],
 ['tank', 'heal', 'dd'],
 ['tank', 'dd'],
 ['tank', 'dd'],
 ['tank', 'heal', 'dd'],
 ['tank', 'dd'],
 ['tank', 'heal', 'dd'],
 ['tank', 'heal', 'dd'],
 ['tank', 'dd']]

In [117]:
# Store largest c found. This will be the number of groups we can create.
n_groups = 0

for i, c in enumerate(itertools.product(*player_roles)):
    
    # Check how many groups could we create of this role combination
    c_candidate = min(c.count('tank'), c.count('heal'), c.count('dd') // 2)
    
    # Store if highest known
    if n_groups < c_candidate:
        n_groups = c_candidate
    
    # Early stopping.
    if i > 10**7:
        break

In [118]:
print("[INFO] According to the first billion rows, we can create this many groups:", n_groups)

[INFO] According to the first billion rows, we can create this many groups: 5


## Product to find potential group compositions

In [128]:
# Store
potential_roles = []

# Stopping condition: 100k
role_store_early_stopping = 10**5

for i, c in enumerate(itertools.product(*player_roles)):
    
    # Check how many groups could we create of this role combination
    c_candidate = min(c.count('tank'), c.count('heal'), c.count('dd') // 2)
    
    if c_candidate == n_groups:
        potential_roles.append(c)
    
    # Early stopping
    if len(potential_roles) > 10**5:
        break

In [129]:
print("[INFO] The first 10 group compositions look like this:")
for i in range(10):
    print(", ".join(potential_roles[i]))

[INFO] The first 10 group compositions look like this:
tank, tank, tank, tank, tank, dd, dd, heal, dd, heal, dd, dd, heal, dd, dd, heal, dd, heal, dd, dd
tank, tank, tank, tank, tank, dd, dd, heal, dd, heal, dd, dd, heal, dd, dd, heal, dd, dd, heal, dd
tank, tank, tank, tank, tank, dd, dd, heal, dd, heal, dd, dd, heal, dd, dd, dd, dd, heal, heal, dd
tank, tank, tank, tank, tank, dd, dd, heal, dd, heal, dd, dd, dd, dd, dd, heal, dd, heal, heal, dd
tank, tank, tank, tank, tank, dd, dd, heal, dd, dd, dd, dd, heal, dd, dd, heal, dd, heal, heal, dd
tank, tank, tank, tank, tank, dd, dd, dd, dd, heal, dd, dd, heal, dd, dd, heal, dd, heal, heal, dd
tank, tank, tank, tank, heal, tank, dd, heal, dd, heal, dd, dd, heal, dd, dd, heal, dd, dd, dd, dd
tank, tank, tank, tank, heal, tank, dd, heal, dd, heal, dd, dd, heal, dd, dd, dd, dd, heal, dd, dd
tank, tank, tank, tank, heal, tank, dd, heal, dd, heal, dd, dd, heal, dd, dd, dd, dd, dd, heal, dd
tank, tank, tank, tank, heal, tank, dd, heal, dd, heal

In [130]:
print("[INFO] The last 10 group compositions look like this:")
for i in range(10):
    print(", ".join(potential_roles[-i]))

[INFO] The last 10 group compositions look like this:
tank, tank, tank, tank, tank, dd, dd, heal, dd, heal, dd, dd, heal, dd, dd, heal, dd, heal, dd, dd
tank, dd, dd, tank, heal, dd, dd, tank, dd, heal, dd, dd, heal, dd, tank, dd, dd, heal, heal, tank
tank, dd, dd, tank, heal, dd, dd, tank, dd, heal, dd, dd, heal, dd, tank, dd, tank, heal, heal, dd
tank, dd, dd, tank, heal, dd, dd, tank, dd, heal, dd, dd, heal, dd, tank, heal, dd, dd, heal, tank
tank, dd, dd, tank, heal, dd, dd, tank, dd, heal, dd, dd, heal, dd, tank, heal, dd, heal, dd, tank
tank, dd, dd, tank, heal, dd, dd, tank, dd, heal, dd, dd, heal, dd, tank, heal, dd, heal, tank, dd
tank, dd, dd, tank, heal, dd, dd, tank, dd, heal, dd, dd, heal, dd, tank, heal, dd, tank, heal, dd
tank, dd, dd, tank, heal, dd, dd, tank, dd, heal, dd, dd, heal, dd, tank, heal, tank, dd, heal, dd
tank, dd, dd, tank, heal, dd, dd, tank, dd, heal, dd, dd, heal, dd, tank, heal, tank, heal, dd, dd
tank, dd, dd, tank, heal, dd, dd, tank, dd, heal, dd, d

## Inspect potential problems

As we can see below, we can end up in a situation where our early stopping condition was too harsh. We do NOT have all possible combinations. Even worse, we do not know how many we are missing.

In [141]:
cond_met = len(potential_roles) > role_store_early_stopping
print("[INFO] The claim that we meat the early stopping condition is: ", cond_met)

[INFO] The claim that we meat the early stopping condition is:  True


## First player inspection

The first player should have a chance of ending up as a DD or a Healer. Is this true? Let's find out how many lottery tickets does this Primus have for various roles.

In [149]:
ttt = 0
ttd = 0
tth = 0

for i in range(len(potential_roles)):
    if potential_roles[i][0] == 'tank':
        ttt +=1
    elif potential_roles[i][0] == 'dd':
        ttd += 1
    else:
        tth += 1
        
print("[INFO] Our Primus has: ")
print(f"{ttt:>6} lottery tickets for tanking")
print(f"{ttd:>6} lottery tickets for tanking")
print(f"{tth:>6} lottery tickets for tanking")

[INFO] Our Primus has: 
100001 lottery tickets for tanking
     0 lottery tickets for tanking
     0 lottery tickets for tanking


## Simple solution

This stops being a problem if we do either or both:
* Shuffle the players list before running the algorithm
* Increase the early stopping condition

Latter is problematic. We might end up in OutOfMemory errors. Thus, let's simply shuffle the players list.

In [152]:
cmax_early_stopping = 10 ** 7
role_store_early_stopping = 10 ** 5

players_copy = players.copy()

random.shuffle(players_copy)

# List of lists. Each list element contains one player's roles as strings.
player_roles = [p.roles for p in players_copy]

# Store largest candidate found. Start value.
n_groups = 0

for i, c in enumerate(itertools.product(*player_roles)):

    # Check how many groups could we create of this role combination
    c_candidate = min(c.count('tank'), c.count('heal'), c.count('dd') // 2)

    # Store if highest known
    if n_groups < c_candidate:
        n_groups = c_candidate

    # Early stopping.
    if i > cmax_early_stopping:
        break

# Container for potential role combinations
potential_roles = []

for i, c in enumerate(itertools.product(*player_roles)):

    # Check how many groups could we create of this role combination
    c_candidate = min(c.count('tank'), c.count('heal'), c.count('dd') // 2)

    # Keep condition
    if c_candidate == n_groups:
        potential_roles.append(c)

    # Early stopping
    if len(potential_roles) > role_store_early_stopping:
        break

# Choose list of roles
chosen_roles = random.choice(potential_roles)

for player, role in zip(players_copy, chosen_roles):
    # Tell Player that it is in roster
    player.reserve(role)

In [165]:
for p in players_copy:
    print(f"{p.name:>10} : {p.chosen_role}")

         q : tank
         p : dd
         m : tank
         n : dd
         k : dd
         j : dd
         h : tank
         g : dd
         a : tank
    primus : heal
         i : dd
         s : dd
         r : heal
         b : heal
         l : dd
         e : dd
         c : dd
         o : heal
         f : tank
         d : heal


## Last step

The last step is to pick 5 tanks, 5 healers and 10 DD's into different groups. Since the list is shuffled, we can simply perform this in order.